In [1]:
from efficient_rerank import run_pipeline, XLMCometEmbeds
import torch
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
import pickle
import pandas as pd
import os
from src.recom_search.model.beam_node_reverse import ReverseNode
import numpy as np
import math 
from new_flatten_lattice import get_dictlist
import flatten_lattice as fl
import json
import ast
import random
xlm_tok = fl.bert_tok


2022-11-26 07:56:09.494796: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-26 07:56:09.494816: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# let's use this as our evaluation metric for diversity
def get_unique_ngrams(sentence, tok, n, uns):
    toks = tok(sentence).input_ids
    #print(toks)
    for i in range(len(toks)-n):
        tmp = ""
        for j in range(i, i+n):
            tmp = tmp+"_"+str(toks[j])
        uns.add(tmp)

def cand_unique_ngrams(sentences, tok, n):
    uniques = set()
    for s in sentences:
        get_unique_ngrams(s, tok, n, uniques)
    return uniques

In [3]:
# TODO something that gets metrics for diverse decoding top k

# load in model, for french
#del model
model = XLMCometEmbeds(drop_rate=0.1)
model.load_state_dict(torch.load("./torchsaved/maskedcont4.pt"))
model.eval()
torch.cuda.memory_allocated(device)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


0

In [3]:
base = "frtest_reversed/"

In [4]:
# get output for a single index out of available graphs
def test_graph_ind(ind, basedir, scofunct):
    g = pickle.load(open(basedir+str(ind), 'rb'))
    #if g['input'] in old['src']:
    #    return None, None, None
    #try:
    global usedlist
    usedlist = []
    options = []
    # TODO add a verbose option for efficient reranking so that it doesn't blow up nb
    return g['input'], g['ref'], run_pipeline(g, model, scofunct, False, 10)

# get predictions for a bunch of stuff
def get_all_preds(basedir, scofunct):
    l = len(os.listdir(basedir))
    result = []
    print("will predict total of ", l)
    for i in range(l):
        inp, r, p = test_graph_ind(i, basedir, scofunct)
        result.append({
            'src':inp,
            'hyp':p,
            'ref':r
        })
        print(i)
    result = pd.DataFrame(result)
    return result

In [5]:
WEIGHT=50
#DIVWEIGHT = 1
#doadd = True
DIVWEIGHT = 1
doadd = True
    
def tokendiverse (node, used):
    if hasattr(node, "prob"):
        #pcnt+=1
        tcnt = [u.token_idx for u in used].count(node.token_idx)
        return math.log(node.prob) + WEIGHT*node.score - DIVWEIGHT*tcnt
    else:
        #npcnt+=1
        return 1
    
def nodediverse (node, used):
    if hasattr(node, "prob"):
        #pcnt+=1
        if node in used:
            if doadd:
                return math.log(node.prob) + WEIGHT*node.score - DIVWEIGHT
        return math.log(node.prob) + WEIGHT*node.score
    else:
        #npcnt+=1
        return 1
    
def extranodediverse (node, used):
    if hasattr(node, "prob"):
        #pcnt+=1
        if node in used:
            return math.log(node.prob) + WEIGHT*node.score - DIVWEIGHT*used.count(node)
        return math.log(node.prob) + WEIGHT*node.score
    else:
        #npcnt+=1
        return 1    


In [75]:
t1 = test_graph_ind(1, base, tokendiverse)

original nodes -  595
final detokd -  616
maxend , 45
maxend , 45
maxend , 46
maxend , 46
maxend , 46
maxend , 46
maxend , 46
maxend , 46
maxend , 46
maxend , 46


In [76]:
t1[2]

['</s> As Iran sought to expand its influence and assert its interests, as well as those of its Shiite allies, discussions with the Council became closely linked to regional ambition.',
 '</s> As Iran sought to extend its influence and assert its interests, along with those of its Shia allies, discussions in the Security Council have become closely tied to regional ambition.',
 '</s> As Iran sought to expand its influence and advance its interests, as well as those of its Shiite allies, discussions in the Security Council became intimately linked with regional ambition.',
 '</s> As Iran sought to extend its influence and assert its interests, along with those of its Shiite allies, talks in the Security Council became closely bound up with regional aspirations.',
 '</s> As Iran sought to expand its influence and advance its interests, as well as those of its Shia allies, discussions in the Security Council have become closely related to regional ambition.',
 '</s> As Iran sought to exte

In [77]:
len(cand_unique_ngrams(t1[2], xlm_tok, 2))

91

In [ ]:
t1[2]

In [ ]:
apreds = get_all_preds(base, tokendiverse)

In [6]:
# token, with stacked weighting - stackweightlatpredsfr-en.csv
# token, no stacked weighting - div1latpredsfr-en.csv
# unrolled with predictions - stackweightcometsfr-en.csv
#apreds.to_csv("stackweightlatpredsfr-en.csv")
apreds = pd.read_csv("stackweightlatpredsfr-en.csv")

In [8]:
ast.literal_eval(apreds.loc[2]['hyp'])

['</s> Another variable is the extent to which other developed countries’ monetary policies will become more flexible.',
 '</s> Another variable is the degree to which other developed countries’ monetary policies will ease.',
 '</s> Another variable is the extent to which monetary policy in other developed economies become more flexible.',
 '</s> Another variable is the degree to which monetary policies in other developed nations will be flexible.',
 '</s> Another variable is the extent to which other developed countries’ monetary policies will ease.',
 '</s> Another variable is the degree to which monetary policy in other advanced economies become looser.',
 '</s> Another variable is the extent to which monetary policies of other advanced countries are becoming more flexible.',
 '</s> Another variable is the degree to which monetary policy in other developed nations will relax.',
 '</s> Another variable is the extent to which monetary policies of other advanced economies become less r

In [7]:
LIM = 3
def evaluate_diversity_all (preds, n):
    totdiv = 0 
    ps = 'hyp'
    if 'hyp' not in preds.keys():
        ps = 'ahyp'
    for h in preds[ps]:
        if type(h) is str:
            hyps = ast.literal_eval(h)
        else:
            hyps = h
        if len(hyps)>LIM:
            hyps = hyps[:LIM]
        #print(len(cand_unique_ngrams(row['hyp'], xlm_tok, 3)))
        #print(len(cand_unique_ngrams(hyps, xlm_tok, 3)))
        totdiv+=len(cand_unique_ngrams(hyps, xlm_tok, n))
    return totdiv / len(preds)

In [8]:
evaluate_diversity_all(apreds, 2)

51.86026200873363

In [9]:
# TODO something that gets metrics for beam search top k
beampreds = pd.read_csv("testbeam50fr_en.csv")

In [10]:
bpclump = beampreds.groupby("ref").agg(list)
    

In [11]:
evaluate_diversity_all(bpclump, 2)

39.7707423580786

In [19]:
# prepare preds for COMET rescoring
def preds_to_complete(preddf):
    res = []
    for index, row in preddf.iterrows():
        hyps = ast.literal_eval(row['hyp'])
        for h in hyps:
            
            res.append({
                'src':row['src'],
                'ref':row['ref'],
                'hyp':h
            })
    return pd.DataFrame(res)

In [20]:
from mt_scores import get_scores_auto

In [26]:
def get_act_hyps(hyplist, cutoff):
    res = []
    for h in hyplist:
        cind = h.index(cutoff)+len(cutoff)
        res.append(h[cind:])
    return res

def get_process_lpreds(preddf):
    lpreds = preddf
    #lpreds['ind'] = lpreds['Unnamed: 0']
    lpreds = lpreds.dropna()
    if "ahyp" not in lpreds.keys():
        lpreds['ahyp'] = get_act_hyps(lpreds['hyp'], "</s>")
    if "comet" not in lpreds.keys():
        comsco = get_scores_auto(lpreds['ahyp'], lpreds['src'], lpreds['ref'], "comet", "")
        lpreds['comet'] = comsco
    return lpreds

In [27]:
predprepped = preds_to_complete(apreds)

In [28]:
newdf = get_process_lpreds(predprepped)

wmt20-comet-da is already in cache.
Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Encoder model frozen.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...


Predicting DataLoader 0:   0%|                                                                              | 0/144 [00:00<?, ?it/s]

To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Predicting DataLoader 0: 100%|████████████████████████████████████████████████████████████████████| 144/144 [00:40<00:00,  3.56it/s]

To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Predicting DataLoader 0: 100%|████████████████████████████████████████████████████████████████████| 144/144 [00:40<00:00,  3.55it/s]


TOOK TIME  41.94


In [30]:
sum(newdf['comet'])/len(newdf['comet'])

0.4513895671456792

In [33]:
sum(beampreds['comet'])/len(beampreds['comet'])

0.6592082606633417

In [34]:
newdf

,src,ref,hyp,ahyp,comet
0,"Après tout, en tant que journaliste d'investig...","After all, as a campaigning investigative jour...","</s> After all, as a field investigative journ...","After all, as a field investigative journalis...",-0.211310
1,"Après tout, en tant que journaliste d'investig...","After all, as a campaigning investigative jour...","</s> After all, as a field investigative journ...","After all, as a field investigative journalis...",0.262921
2,"Après tout, en tant que journaliste d'investig...","After all, as a campaigning investigative jour...","</s> After all, as an investigative journalist...","After all, as an investigative journalist on ...",-0.289604
3,"Après tout, en tant que journaliste d'investig...","After all, as a campaigning investigative jour...","</s> After all, as a field investigative journ...","After all, as a field investigative journalis...",0.326253
4,"Après tout, en tant que journaliste d'investig...","After all, as a campaigning investigative jour...","</s> After all, as an investigative reporter i...","After all, as an investigative reporter in th...",-0.203522
...,...,...,...,...,...
4575,"Au plan intérieur également, une méfiance géné...","At home, too, massive distrust will further co...","</s> At home, too, widespread mistrust would c...","At home, too, widespread mistrust would compl...",0.784285
4576,"Au plan intérieur également, une méfiance géné...","At home, too, massive distrust will further co...","</s> Internally, too, a generalized mistrust w...","Internally, too, a generalized mistrust will ...",0.231514
4577,"Au plan intérieur également, une méfiance géné...","At home, too, massive distrust will further co...","</s> At home, too, widespread distrust will ad...","At home, too, widespread distrust will add to...",0.695930
4578,"Au plan intérieur également, une méfiance géné...","At home, too, massive distrust will further co...","</s> Internally, too, a generalized mistrust w...","Internally, too, a generalized mistrust would...",0.687333
